# ElasticSearch + эвристики
Или берём лучшее от двух предыдущих версий стандартизатора адресов

In [394]:
import pandas as pd
import random
import re
from elasticsearch import Elasticsearch as es
pd.options.display.max_columns = None
import itertools,operator

In [397]:
'''
Не работает без Elastic с загруженным туда ФИАС и проиндексированным на поиск родителей каждой строки
'''
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [251]:
"""
Словарь, который используется для детекции номера дома, корпуса и т.д.
"""
sep_house_signs = {
  'дом': {'д', 'дом'},
  'владение':{'владение', 'вл'},
  'корпус': {'к', 'корп', 'копр', 'кор', 'корпус'},
  'строение':{'с', 'стр', 'строен', 'строение'},
  'квартира': {'кв', 'квартира'},
  'помещение':{'пом', 'помещение'},
  'комната':{"ком",'комн', 'комната'},
  'кабинет':{"кабинет", "каб", "к-т", "каб-т"},
  'офис': {'оф', 'офис'},
  'литера': set("абвежз"),
  'прочее': {'литер', 'литера', 'лит'}
}

In [252]:
house_signs_inv = inverdic(sep_house_signs)

In [445]:
def verify_address(full_address, debug=False):
  '''
  Ищет адрес в ФИАС
  Вход: строка
  Выход: словарь с полным адресом по ФИАС и его составляющими
  '''
  if full_address == '':
    return []
  query = {
  'size':1,
  "query":{
    "query_string" : {
            "fields" : ["fullname"],
            "query" : preprocess_for_search(full_address),
            "fuzziness": "auto",
            "use_dis_max" :  "true"
        }
  }
  }
  response = es.search(index='fias_full_text', doc_type='address', body=query)
  try:
    return response["hits"]["hits"][0]["_source"]
  except IndexError:
    return []
#   # Найти номер дома в ФИАС
#   if dic.get('Улица', False):
#     response.update(verify_home(dic, response['guid']))
#   response.update({'original':dic['original']})
#   if not debug:
#     try:
#       return response
#     except:
#       return []
#   else:
#     return response

In [384]:
def del_sp_char(string):
  '''
  Стоплист. Удаляет из строки символы переноса строки, но словарь можно дополнить при надобности
  '''
  for stopword in {r'\n', r'\r', '\\', '/', '(', ')', ':'}:
    string = string.replace(stopword, ' ')
  return re.sub(r"[\d]+", ' \g<0> ', string)

In [407]:
def preprocess(string):
  string = del_sp_char(string)
  string = re.sub(r"[\d]+", ' \g<0> ', string)
  string = string.replace(',', ', ')
  string = re.sub(r' +', ' ', string)
  return string

In [ ]:
def preprocess_for_search(string):
  
  return string

In [258]:
def extract_index(string, errors=False):   # 100% works !!!
  '''
  Извлекает индекс из строки
  Вход: строка
  Выход: адрес без индекса, индекс
  '''
  index = re.findall(r'[^| |,][\d]{5}[ |$|, ]', string)
  if len(index) > 1 and errors:
    print("Два индекса в строке \"%s\" ?"%string)

  if index != []:
    index = index[0]
    string = string.replace(index, '').strip()
    index = index.replace(',', '')
  else:
    index = None
  return string, index

In [395]:
def tokenize(string, comma=False):
  '''
  Токенизатор. Раздвигает слипшиеся буквы и цифры вроде 2с3 или корп1
  Вход: строка
  Выход: массив из слов (токены)
  '''
  # string = re.sub(r"[\d]+", ' \g<0> ', string).lower() — это уже сделано при препроцессинге
  string = string.lower()
  if not comma:
    return re.findall(r'[\d]+|[\w]+', string)
  if comma:
    return re.findall(r'[\d]+|[\w]+|\,', string)

## Извлечение дома

In [365]:
def extract_house_tokens(tokens):
  a = lambda x: "число" if x.isdigit() and len(x)<6 else "препинания" if x == ',' else "не распознано"
  types = [house_signs_inv.get(x, a(x)) for x in tokens]
  types_bin = [0 if x=='не распознано' else 1  for x in types]
  array = list((list(y) for (x,y) in itertools.groupby((enumerate(types_bin)),operator.itemgetter(1)) if x == 1))
  if len(array) == 0:
    return [], []
  longest_seq = max(reversed(array), key=len)
  return [tokens[i] for (i, _) in longest_seq], [types[i] for (i, _) in longest_seq]

In [334]:
def extract_house(string): #from 2.0
  '''
  Обёртка для процедуры извлечения номера дома/корпуса от оставшейся строки
  Вход: адрес(строка)
  Выход: адрес без номеров дома/корпуса, номера дома/корпуса строкой
  '''
  tokens = tokenize(string, comma=True)
  house_tokens, house_types = extract_house_tokens(tokens)
  
  pattern = r".?.?".join(house_tokens)
  found = re.search(pattern, string.lower())
  if found==None:
    split = len(string)
  else:
    split = found.start()
  #split = re.search(pattern, string.lower()).start()
  address = string[:split]
  house = string[split:]
  return address, house

In [387]:
def search_dict(word, dic):
  '''
  Поиск по словарю. Ищет по всем вложенным рубрикам словаря и возвращает путь к найденному слову
  Вход: токен (должен быть написан маленькими буквами) и опционально словарь
  Выход: Путь к слову/несколько путей
  '''
  if isinstance(dic, dict):
    winners = []
    for key, value in dic.items():
      response = search_dict(word, value)
      if response == True:
        winners.append(key)
      elif isinstance(response, list):
        for instance in response:
          if isinstance(instance, list):
            winners.append([key, *instance])
          else:
            winners.append([key, instance])
      elif isinstance(response, str):
        winners.append([key, response])
    return winners
  elif isinstance(dic, set) or isinstance(dic, list):
    if word in dic:
      return True

In [442]:
def standardize(string, origin = True):
  '''
  Обёртка для всех методов выше. Разделяет адрес на его составляющие и ищет совпадение в ФИАС. В 90+% случаев находит.
  Вход: строка с адресом
  Выход: составляющие адреса
  '''
  dic = {}
  if origin:
    dic['origin'] = string
  string = preprocess(string)
  address, index = extract_index(string)
  address, house = extract_house(address)
  dic['index']=index
  dic['address'] = address
  dic.update(verify_address(address))
  dic['house'] = house
  return dic

In [443]:
def get_addr(strings, progress=True):
  '''
  Обрабатывает несколько адресов подряд.
  Вход: массив строк
  Выход: pandas Dataframe
  '''
  dics = []
  n = len(strings) - 1
  for i,  line in enumerate(strings):
    if progress:
      print("Working on {0} of {1}. Progress {2:03.1f}%".format(i, n, (i/n)*100), end='\r')
    dics.append(standardize(line))
  return pd.DataFrame(dics)

## Методы для контроля качества

In [433]:
def score(ref, orig_col="Исходный адрес", func = get_addr, cols_to_score = ["Регион", "Район", "Город", "Н/п", "Улица"]):
  df_1 = func(ref[orig_col]).rename(index=str, columns=dadata_LUT)
  ref, df_1 = ref.fillna(''), df_1.fillna('')
  #### Не проверялось, так что может не работать
  df_1 = df_1.to_dict(orient='records')
  N = ref.shape[0]
  ref=ref.to_dict(orient='records')
  n = 0
  correct = 0
  df = []
  if len(df_1) != len(ref):
    return "не совпадают размеры таблицы"
  for i, row in enumerate(ref):
    for key, value in row.items():
      if key in cols_to_score:   #["Регион", "Индекс", "Район", "Город", "Н/п", "Улица", "Дом", "Корпус/строение"]:
        n += 1
        if value == df_1[i][key]:
          correct += 1
        else:
          df.append(df_1[i])
  print("\n{0:03.1f}% correct fields".format(correct/n*100))
  df = pd.DataFrame(df).drop_duplicates()
  print("\n{0:03.1f}% correct lines".format((N-df.shape[0])/N*100))
  return df.sample(20)

In [269]:
def show_changes(df1, df2):
  '''
  Выводит список изменений между df1 и df2
  '''
  if df1.shape != df2.shape:
    print("Разные таблицы")
    return
  df1, df2 = df1.fillna(''), df2.fillna('')
  delta_cols = list(df1.columns[(df1 != df2).any(0)])
  print("Изменились столбцы " + ", ".join(delta_cols))
  n = 0
  for col in delta_cols:
    diffs = df1[col] != df2[col]
    changes = set()
    n_i = 0
    for index, changed in diffs.iteritems():
      if changed:
        out = str(df1.iloc[index][col]) + ' ==> ' + str(df2.iloc[index][col])
        n += 1
        n_i +=1
        if out not in changes:
          changes.add(out)
          print(out)
    print("{} изменений в столбце".format(n_i))
  print("Всего изменено {} значений".format(n))

In [431]:
dadata_LUT = {'original':'Исходный адрес',
  'fullname':'Адрес',
  'postalcode':'Индекс',
  'country':'Страна',
  'region_type':'Тип региона',
  'region':'Регион',
  'area_type':'Тип района',
  'area':'Район',
  'city_type':'Тип города',
  'city':'Город',
  '65_type':'Тип н/п',
  '65':'Н/п',
  '???':'Адм. округ',
  'town':'Н/п',
  'town_type':'Тип н/п',
  'district_type':'Тип района',
  'district':'Район города',
  'street_type':'Тип улицы',
  'street':'Улица',
  'house_type':'Тип дома',
  'housenum':'Дом',
  'build_type':'Тип корпуса/строения',
  'buildnum':'Корпус/строение',
  'struc_type':'Тип корпуса/строения',
  'strucnum':'Корпус/строение',            
  'flat_type':'Тип квартиры',
  'flat_num':'Номер Квартиры'
}
# def rename_fias(dic, LUT=dadata_LUT):
#   return {LUT.get(key, key):value for key, value in dic.items() if key not in ['aolevel','guid', 'houseid', 'district_type']}

### TODO:
1. отцеплять адреса по 2.0 технологии
2. Дробить их по ней же
3. Отправлять в Elastic 
4. сверять индекс. Если вышла шляпа — попытаться пофиксить (хз как пока, но как вариант второй адрес из выдачи или увеличить ~)
5. Добавить критерий оценки качества: если длинна строки большая, а AOLEVEL < 3, то 99% что это дичь

## Проверка работоспособности

In [270]:
cols2 = ['origin',
 'Регион',
 'Район',
 'Город',
 'Н/п',
 'Улица']

In [399]:
ref  = pd.read_excel('ref/sj.xlsx')
ref2 = pd.read_excel('ref/references.xlsx')

In [444]:
new_method_mistakes = score(ref2.sample(300))
new_method_mistakes

Working on 1040 of 1040. Progress 100.0%
88.5% correct fields

75.7% correct lines


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """


,address,aolevel,guid,house,index,origin,Адрес,Город,Н/п,Район,Регион,Тип города,Тип н/п,Тип района,Тип региона,Тип улицы,Улица
271,", , Россия, Москва г, Зеленоград г, к. 1606 , ...",65,71559100-fb16-47df-9ed8-9e9145397aa5,,,", , Россия, Москва г, Зеленоград г, к. 1606, к...","г Москва, п Марушкинское, кв-л 79",,,Марушкинское,Москва,,,п,г,,
269,"Иное, , Россия, Дагестан Респ, Кизляр г, Куйбы...",4,e18e67f7-e62a-4abe-853c-ffb8e5ab57e1,д. 8,368832,"Иное, 368832, Россия, Дагестан Респ, Кизляр г,...","Респ Дагестан, г Кизляр",Кизляр,,,Дагестан,г,,,Респ,,
353,"Москва, , Россия, Москва г, Ротерта ул, д. 9 ,...",65,68b68cd8-91f3-432e-b120-af4556a6a831,,129347,"Москва, 129347, Россия, Москва г, Ротерта ул, ...","г Москва, п Сосенское, кв-л 182",,,Сосенское,Москва,,,п,г,,
260,"Москва, , Россия, Москва г, Братеевская ул, д....",65,42f79b42-40a0-41bd-8727-8c386e8e708c,,,"Москва, , Россия, Москва г, Братеевская ул, д....","г Москва, п Новофедоровское, кв-л 272",,,Новофедоровское,Москва,,,п,г,,
85,", , Россия, Москва г, Братиславская ул, д. 22 ...",65,0900d68c-73b4-455d-860a-3ce0dd2c2570,,,", , Россия, Москва г, Братиславская ул, д. 22,...","г Москва, п Марушкинское, кв-л 117",,,Марушкинское,Москва,,,п,г,,
480,"МОСКВА, , ул. Рязанский, д. 86 1 , стр. 3 , оф...",7,852d0bfb-f542-450a-94ed-360697c8fc4c,,109542,"МОСКВА, 109542, ул. Рязанский, д. 86/1, стр. 3...","край Хабаровский, г Хабаровск, стр Гск-86",Хабаровск,,,Хабаровский,г,,,край,стр,Гск-86
393,,,,Москва,,Москва,,,,,,,,,,,
11,", ОКТЯБРЬСКИЙ , , КОМСОМОЛЬСКАЯ,",7,9c390dc8-5c8b-4339-ac31-bacfeb299c1e,д. 101,346480,"346480, ОКТЯБРЬСКИЙ , , КОМСОМОЛЬСКАЯ,д.101","обл Волгоградская, р-н Октябрьский, рп Октябрь...",,Октябрьский,Октябрьский,Волгоградская,,рп,р-н,обл,ул,Комсомольская
16,", САЛЬСКИЙ , , ЧЕРЕМУШКИ,",7,8f171b8f-67db-4224-9a13-59ec9f76e754,д. 20 2,347629,"347629, САЛЬСКИЙ , , ЧЕРЕМУШКИ,д.20 2","обл Ростовская, р-н Сальский, г Сальск, ул Чер...",Сальск,,Сальский,Ростовская,г,,р-н,обл,ул,Черемушки
376,"Москва, , Россия, Москва г, Тушинский 1 -й про...",65,2bab4b4f-228d-4e60-a364-9c8eac79ce63,,125371,"Москва, 125371, Россия, Москва г, Тушинский 1-...","г Москва, п Роговское, кв-л 92",,,Роговское,Москва,,,п,г,,


In [361]:
Out[354].loc[244].origin

'141200, Российская Федерация, 50 Московская область, Пушкино, Московский проспект, 57'

In [366]:
standardize(ref2['Исходный адрес'].iloc[random.randint(0, 1040)])

{'address': ', , Россия, Москва г, Новокосинская ул',
 'house': ', д. 21 , кв. 27 ',
 'index': None,
 'origin': ', , Россия, Москва г, Новокосинская ул, д. 21, кв. 27',
 'Адрес': 'г Москва, ул Новокосинская',
 'Регион': 'Москва',
 'Тип региона': 'г',
 'Тип улицы': 'ул',
 'Улица': 'Новокосинская'}

In [703]:
ref2['Исходный адрес'].sample(1).iloc[0]

'125581, Москва г, Ляпидевского ул, д. 2, кв. 9'

In [332]:
get_addr(ref2['Исходный адрес'])

,address,house,index,origin,Адрес,Город,Н/п,Район,Регион,Тип города,Тип н/п,Тип района,Тип региона,Тип улицы,Улица
0,", Неклиновский р-н х Красный Десант, ,ул Октяб...",",д. 74 А",346841,"346841 , Неклиновский р-н х Красный Десант, ,...","обл Ростовская, р-н Неклиновский, х Красный Де...",NaN,Красный Десант,Неклиновский,Ростовская,NaN,х,р-н,обл,ул,Октябрьская
1,", г Новошахтинск, ,ул Харьковская,д. 14 10",,346900,"346900 , г Новошахтинск, ,ул Харьковская,д. 1...","обл Ростовская, г Новошахтинск, ул Харьковская",Новошахтинск,NaN,NaN,Ростовская,г,NaN,NaN,обл,ул,Харьковская
2,", г Ростов-на-Дону, ,пр-кт Коммунистический,д....",,344000,"344000 , г Ростов-на-Дону, ,пр-кт Коммунистич...","обл Ростовская, г Ростов-на-Дону, пр-кт Коммун...",Ростов-на-Дону,NaN,NaN,Ростовская,г,NaN,NaN,обл,пр-кт,Коммунистический
3,", Азовский р-н с Самарское, ,пер Рабочий",",д. 13",346751,"346751 , Азовский р-н с Самарское, ,пер Рабоч...","обл Ростовская, р-н Азовский, с Самарское, пер...",NaN,Самарское,Азовский,Ростовская,NaN,с,р-н,обл,пер,Рабочий
4,", Семикаракорский р-н п Вершинный, ,ул Сельская",",д. 16",346641,"346641 , Семикаракорский р-н п Вершинный, ,ул...","обл Ростовская, р-н Семикаракорский, п Вершинн...",NaN,Вершинный,Семикаракорский,Ростовская,NaN,п,р-н,обл,ул,Сельская
5,", Октябрьский р-н п Новосветловский, ,пер Доне...",",д. 14",346480,"346480 , Октябрьский р-н п Новосветловский, ,...","обл Ростовская, р-н Октябрьский, п Новосветлов...",NaN,Новосветловский,Октябрьский,Ростовская,NaN,п,р-н,обл,пер,Донецкий
6,", Красносулинский р-н г Красный Сулин, ,ул Бол...",",д. 4 В 35",346352,"346352 , Красносулинский р-н г Красный Сулин,...","обл Ростовская, р-н Красносулинский, г Красный...",Красный Сулин,NaN,Красносулинский,Ростовская,г,NaN,р-н,обл,ул,Больничная
7,", г Таганрог, ,ул Акушерская",",д. 69",347900,"347900 , г Таганрог, ,ул Акушерская,д. 69","обл Ростовская, г Таганрог, ул Акушерская",Таганрог,NaN,NaN,Ростовская,г,NaN,NaN,обл,ул,Акушерская
8,", г Таганрог, ,ул Циолковского,д. 32 2 72",,347900,"347900 , г Таганрог, ,ул Циолковского,д. 32 ...","обл Ростовская, г Таганрог, ул Циолковского",Таганрог,NaN,NaN,Ростовская,г,NaN,NaN,обл,ул,Циолковского
9,", Азовский п Тимирязево, ,ул Кольцевя,д. 1 16",,346746,"346746 , Азовский п Тимирязево, ,ул Кольцевя,...","обл Калининградская, р-н Славский, п Тимирязев...",NaN,Тимирязево,Славский,Калининградская,NaN,п,р-н,обл,ул,1 Мая


In [470]:
ref2['Исходный адрес'][:30]

0     346841, Неклиновский р-н х Красный Десант, ,ул...
1       346900, г Новошахтинск, ,ул Харьковская,д.14 10
2     344000, г Ростов-на-Дону, ,пр-кт Коммунистичес...
3     346751, Азовский р-н с Самарское, ,пер Рабочий...
4     346641, Семикаракорский р-н п Вершинный, ,ул С...
5     346480, Октябрьский р-н п Новосветловский, ,пе...
6     346352, Красносулинский р-н г Красный Сулин, ,...
7               347900, г Таганрог, ,ул Акушерская,д.69
8        347900, г Таганрог, ,ул Циолковского,д.32/2 72
9     346746, Азовский п Тимирязево, ,ул Кольцевя,д....
10            347900, г Таганрог, ,пер Каркасный,д.5 31
11    346647, Семикаракорский р-н п Крымский, ,ул Ле...
12    346645, Семикаракорский рн х Золотаревка, ,ул ...
13    346521, Октябрьский рн п Новокадамово, ,ул Гаг...
14    347787, Веселовский х В-СОЛЕНЫЙ, , ЦЕНТРАЛЬНАЯ...
15                346974, г с Ряженое, ,пер Чкалова,д.1
16    346500, г Шахты, ,пр-кт Победа Революции,д.120 44
17            344019, РОСТОВ-НА-ДОНУ, , 23 ЛИНИЯ

In [499]:
ref2.iloc[11]['Исходный адрес'].replace('-', '\-')

'346647, Семикаракорский р\\-н п Крымский, ,ул Ленина,д.43 2'

In [711]:
addrs = []
for i, line in ref2['Исходный адрес'].iteritems():
  address, _ = extract_index(ref2['Исходный адрес'].iloc[random.randint(0, 1040)])
  address, _ = extract_house(address)
  addrs.append(address)
addrs

['г Шахты, ,ул Искра,',
 'Москва, , Россия, Москва г, Островитянова ул, ',
 'Москва г, Новокосинская ул, ',
 ',  Россия, Москва г, Хлобыстова ул, ',
 'Российская Федерация, г.Москва ул.Кадомцева ',
 'г Ростов-на-Дону, ,пр-кт Михаила Нагибина,',
 ', , Россия, Московская, Подольский р-н, Подольск г, Давыдова ул, ',
 'Москва, , Россия, Москва г, Винокурова ул, ',
 'Москва,  Россия, г. Москва, Твардовского, ',
 'г. Москва ул. Профсоюзная, ',
 'г. Балаково, Саратовской обл., ул. Коммунистическая, ',
 'Москва г, Михайловское с, ',
 'Москва, , Россия, Москва г, Зеленодольская ул, ',
 'Московская обл, Люберцы г, Победы пр-кт, ',
 'г Новочеркасск, ,ул Александровская,',
 'г Волгодонск, ,ул 50 лет СССР,',
 'г Ростов-на-Дону, ,ул Новаторов,',
 ', , Россия, Москва г, Братиславская ул, ',
 'г. Омск, ул. 27 Северная, ',
 'Москва, , Россия, Москва г, Барышиха ул, ',
 'Москва г, Очаковская Б. ул, ',
 'МОСКВА Г, ОЛЕНЬЯ Б. УЛ, д. 15, кв. В\\Ч',
 'Октябрьский рн п Новокадамово, ,ул Гагарина,',
 'Российск

In [718]:
a=random.choice(addrs)
a

', , Россия, Москва, Москва, Новочеремушкинска, '

In [740]:
a

', , Россия, Москва, Москва, Новочеремушкинска, '

In [741]:
len(addrs)

1041

Working on 965 of 1040. Progress 92.8%

TypeError: unhashable type: 'dict'

In [329]:
Out

{9: [],
 10: ['корпус'],
 11: ['офис'],
 12: ['дом'],
 16: {'а': {'литера'},
  'б': {'литера'},
  'в': {'литера'},
  'вл': {'владение'},
  'владение': {'владение'},
  'д': {'дом'},
  'дом': {'дом'},
  'е': {'литера'},
  'к': {'корпус'},
  'кабинет': {'квартира'},
  'кв': {'квартира'},
  'квартира': {'квартира'},
  'ком': {'комната'},
  'комн': {'комната'},
  'комната': {'комната'},
  'копр': {'корпус'},
  'кор': {'корпус'},
  'корп': {'корпус'},
  'корпус': {'корпус'},
  'лит': {'прочее'},
  'литер': {'прочее'},
  'литера': {'прочее'},
  'оф': {'офис'},
  'офис': {'офис'},
  'пом': {'помещение'},
  'помещение': {'помещение'},
  'с': {'строение'},
  'стр': {'строение'},
  'строен': {'строение'},
  'строение': {'строение'}},
 17: {'литера'},
 18: {'владение'},
 24: {'а': {'литера'},
  'б': {'литера'},
  'в': {'литера'},
  'вл': {'дом'},
  'г': {'город'},
  'гор': {'город'},
  'город': {'город'},
  'д': {'дом'},
  'д-ня': {'деревня'},
  'дер': {'деревня'},
  'деревня': {'деревня'},
  'дом

In [331]:
Out[290]

,address,house,index,origin
0,"Неклиновский р-н х Красный Десант, ,ул Октябрь...",",д.74 А",346841,"346841, Неклиновский р-н х Красный Десант, ,ул..."
1,"г Новошахтинск, ,ул Харьковская",",д.14 10",346900,"346900, г Новошахтинск, ,ул Харьковская,д.14 10"
2,"г Ростов-на-Дону, ,пр-кт Коммунистический",",д.43 165",344000,"344000, г Ростов-на-Дону, ,пр-кт Коммунистичес..."
3,"Азовский р-н с Самарское, ,пер Рабочий",",д.13",346751,"346751, Азовский р-н с Самарское, ,пер Рабочий..."
4,"Семикаракорский р-н п Вершинный, ,ул Сельская",",д.16",346641,"346641, Семикаракорский р-н п Вершинный, ,ул С..."
5,"Октябрьский р-н п Новосветловский, ,пер Донецкий",",д.14",346480,"346480, Октябрьский р-н п Новосветловский, ,пе..."
6,"Красносулинский р-н г Красный Сулин, ,ул Больн...",",д.4В 35",346352,"346352, Красносулинский р-н г Красный Сулин, ,..."
7,"г Таганрог, ,ул Акушерская",",д.69",347900,"347900, г Таганрог, ,ул Акушерская,д.69"
8,"г Таганрог, ,ул Циолковского",",д.32/2 72",347900,"347900, г Таганрог, ,ул Циолковского,д.32/2 72"
9,"Азовский п Тимирязево, ,ул Кольцевя",",д.1 16",346746,"346746, Азовский п Тимирязево, ,ул Кольцевя,д...."


In [531]:
get_addr(ref2['Исходный адрес'][:20])

,address,house,index,origin,Адрес,Город,Н/п,Район,Регион,Тип города,Тип н/п,Тип района,Тип региона,Тип улицы,Улица
0,"Неклиновский р-н х Красный Десант, ,ул Октябрь...",д.74 А,346841,"346841, Неклиновский р-н х Красный Десант, ,ул...","обл Ростовская, р-н Неклиновский, х Красный Де...",NaN,Красный Десант,Неклиновский,Ростовская,NaN,х,р-н,обл,ул,Октябрьская
1,"г Новошахтинск, ,ул Харьковская,",д.14 10,346900,"346900, г Новошахтинск, ,ул Харьковская,д.14 10","обл Ростовская, г Новошахтинск, ул Харьковская",Новошахтинск,NaN,NaN,Ростовская,г,NaN,NaN,обл,ул,Харьковская
2,"г Ростов-на-Дону, ,пр-кт Коммунистический,",д.43 165,344000,"344000, г Ростов-на-Дону, ,пр-кт Коммунистичес...","обл Ростовская, г Ростов-на-Дону, пр-кт Коммун...",Ростов-на-Дону,NaN,NaN,Ростовская,г,NaN,NaN,обл,пр-кт,Коммунистический
3,"Азовский р-н с Самарское, ,пер Рабочий,",д.13,346751,"346751, Азовский р-н с Самарское, ,пер Рабочий...","обл Ростовская, р-н Азовский, с Самарское, пер...",NaN,Самарское,Азовский,Ростовская,NaN,с,р-н,обл,пер,Рабочий
4,"Семикаракорский р-н п Вершинный, ,ул Сельская,",д.16,346641,"346641, Семикаракорский р-н п Вершинный, ,ул С...","обл Ростовская, р-н Семикаракорский, п Вершинн...",NaN,Вершинный,Семикаракорский,Ростовская,NaN,п,р-н,обл,ул,Сельская
5,"Октябрьский р-н п Новосветловский, ,пер Донецкий,",д.14,346480,"346480, Октябрьский р-н п Новосветловский, ,пе...","обл Ростовская, р-н Октябрьский, п Новосветлов...",NaN,Новосветловский,Октябрьский,Ростовская,NaN,п,р-н,обл,пер,Донецкий
6,"Красносулинский р-н г Красный Сулин, ,ул Больн...",д.4В 35,346352,"346352, Красносулинский р-н г Красный Сулин, ,...","обл Ростовская, р-н Красносулинский, г Красный...",Красный Сулин,NaN,Красносулинский,Ростовская,г,NaN,р-н,обл,ул,Больничная
7,"г Таганрог, ,ул Акушерская,",д.69,347900,"347900, г Таганрог, ,ул Акушерская,д.69","обл Ростовская, г Таганрог, ул Акушерская",Таганрог,NaN,NaN,Ростовская,г,NaN,NaN,обл,ул,Акушерская
8,"г Таганрог, ,ул Циолковского,",д.32/2 72,347900,"347900, г Таганрог, ,ул Циолковского,д.32/2 72","обл Ростовская, г Таганрог, ул Циолковского",Таганрог,NaN,NaN,Ростовская,г,NaN,NaN,обл,ул,Циолковского
9,"Азовский п Тимирязево, ,ул Кольцевя,",д.1 16,346746,"346746, Азовский п Тимирязево, ,ул Кольцевя,д....","обл Калининградская, р-н Багратионовский, п Ти...",NaN,Тимирязево,Багратионовский,Калининградская,NaN,п,р-н,обл,NaN,NaN


In [532]:
ref2[:20]

,Исходный адрес,Адрес,Индекс,Страна,Тип региона,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Адм. округ,Район города,Тип улицы,Улица,Тип дома,Дом,Тип корпуса/строения,Корпус/строение,Тип квартиры,Номер Квартиры
0,"346841, Неклиновский р-н х Красный Десант, ,ул...","Ростовская обл, Неклиновский р-н, х Красный Де...",346844.0,Россия,обл,Ростовская,р-н,Неклиновский,NaN,NaN,х,Красный Десант,NaN,NaN,ул,Октябрьская,д,74А,NaN,NaN,NaN,NaN
1,"346900, г Новошахтинск, ,ул Харьковская,д.14 10","Ростовская обл, г Новошахтинск, ул Харьковская...",346900.0,Россия,обл,Ростовская,NaN,NaN,г,Новошахтинск,NaN,NaN,NaN,NaN,ул,Харьковская,д,14,NaN,NaN,кв,10
2,"344000, г Ростов-на-Дону, ,пр-кт Коммунистичес...","г Ростов-на-Дону, пр-кт Коммунистический, д 43...",344091.0,Россия,обл,Ростовская,NaN,NaN,г,Ростов-на-Дону,NaN,NaN,NaN,NaN,пр-кт,Коммунистический,д,43,NaN,NaN,кв,165
3,"346751, Азовский р-н с Самарское, ,пер Рабочий...","Ростовская обл, Азовский р-н, с Самарское, пер...",346751.0,Россия,обл,Ростовская,р-н,Азовский,NaN,NaN,с,Самарское,NaN,NaN,пер,Рабочий,д,13,NaN,NaN,NaN,NaN
4,"346641, Семикаракорский р-н п Вершинный, ,ул С...","Ростовская обл, Семикаракорский р-н, п Вершинн...",346654.0,Россия,обл,Ростовская,р-н,Семикаракорский,NaN,NaN,п,Вершинный,NaN,NaN,ул,Сельская,д,16,NaN,NaN,NaN,NaN
5,"346480, Октябрьский р-н п Новосветловский, ,пе...","Ростовская обл, Октябрьский р-н, п Новосветлов...",346481.0,Россия,обл,Ростовская,р-н,Октябрьский,NaN,NaN,п,Новосветловский,NaN,NaN,пер,Донецкий,д,14,NaN,NaN,NaN,NaN
6,"346352, Красносулинский р-н г Красный Сулин, ,...","Ростовская обл, г Красный Сулин, ул Больничная...",346353.0,Россия,обл,Ростовская,р-н,Красносулинский,г,Красный Сулин,NaN,NaN,NaN,NaN,ул,Больничная,д,4В,NaN,NaN,кв,35
7,"347900, г Таганрог, ,ул Акушерская,д.69","Ростовская обл, г Таганрог, ул Акушерская, д 69",347905.0,Россия,обл,Ростовская,NaN,NaN,г,Таганрог,NaN,NaN,NaN,NaN,ул,Акушерская,д,69,NaN,NaN,NaN,NaN
8,"347900, г Таганрог, ,ул Циолковского,д.32/2 72","Ростовская обл, г Таганрог, ул Циолковского, д...",347916.0,Россия,обл,Ростовская,NaN,NaN,г,Таганрог,NaN,NaN,NaN,NaN,ул,Циолковского,д,32/2,NaN,NaN,кв,72
9,"346746, Азовский п Тимирязево, ,ул Кольцевя,д....","Ростовская обл, Азовский р-н, п Тимирязевский,...",346746.0,Россия,обл,Ростовская,р-н,Азовский,NaN,NaN,п,Тимирязевский,NaN,NaN,ул,Кольцевая,д,1,NaN,NaN,кв,16


In [278]:
ref3 = ref2.sample(1000)
score(get_addr(ref3['Исходный адрес']).fillna(''), ref3.fillna(''))[cols2]

Working on 999 of 999. Progress 100.0%
94.1%


NameError: name 'cols2' is not defined

In [753]:
ref3.iloc[280]

Исходный адрес          344114, г Ростов-на-Дону, ,ул Орбитальная,д.54...
Адрес                      г Ростов-на-Дону, ул Орбитальная, д 54, кв 100
Индекс                                                             344114
Страна                                                             Россия
Тип региона                                                           обл
Регион                                                         Ростовская
Тип района                                                            NaN
Район                                                                 NaN
Тип города                                                              г
Город                                                      Ростов-на-Дону
Тип н/п                                                               NaN
Н/п                                                                   NaN
Адм. округ                                                            NaN
Район города                          

In [414]:
old_method = get_addr(ref2['Исходный адрес'])

In [279]:
cols = ['Исходный адрес',
 'Регион',
 'Район',
 'Город',
 'Н/п',
 'Адм. округ',
 'Район города',
 'Улица']

In [736]:
%timeit verify_address(" г ТАГАНРОГ, ,ул Б. БУЛЬВАРНАЯ", debug=True)

20.8 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [682]:
ref3.iloc[16]

Исходный адрес                  346512, г Шахты, ,пер Цуканова,д.25
Адрес                   Ростовская обл, г Шахты, пер Цуканова, д 25
Индекс                                                       346512
Страна                                                       Россия
Тип региона                                                     обл
Регион                                                   Ростовская
Тип района                                                      NaN
Район                                                           NaN
Тип города                                                        г
Город                                                         Шахты
Тип н/п                                                         NaN
Н/п                                                             NaN
Адм. округ                                                      NaN
Район города                                                    NaN
Тип улицы                                       

In [668]:
shortables = {
  "малая":{"м", "мал", "м-я", "малая"},
  "большая":{"б", "бол", "б-я", "большая"},
  "малый":{"м", "мал", "м-й", "малый"},
  "средний":{"с", "ср", "ср-ий", "ср-й", "средний"},
  "большой":{"б", "бол", "б-й", "большой"},
}
def fuzzy_street_size(string):
  
  return

In [86]:
def inverdic(dic):
  resdic = {}
  for key, value in dic.items():
    for index in value:
      if type(value) == set or type(value) == list:
        if index in resdic.keys():
          if isinstance(resdic[index], set):
            resdic[index].add(key)
          elif isinstance(resdic[index], list):
            resdic[index].append(key)
          else:
            resdic[index] = [resdic[index]]
            resdic[index].append(key)
        else:
          resdic[index] = key
      elif type(value) == dict:
        resdic.update(inverdic(value))
  return resdic

In [87]:
types = {
  'город':{
    'город':{'г', 'гор', 'город'},
    'село':{'с', 'село', 'сел'},
    'деревня':{'д', 'дер', 'деревня', 'д-ня'},
    'с/п':{"сельский", "поселок", "сп"}
  },
  'дом':{
    'дом': {'вл', 'д', 'дом'},
    'корпус': {'к', 'корп', 'копр', 'кор', 'корпус', 'с', 'стр', 'строен', 'строение'},
    'квартира': {'кабинет', 'кв', 'квартира', "ком",'комн', 'пом', 'помещение', 'комната'},
    'офис': {'оф', 'офис'},
    'литера': {'а', 'б', 'в'}
  },
  'страна':{'страна', 'стр'},
  'индекс':{'индекс', 'инд'}
}

In [88]:
inverdic(types)

{'а': 'литера',
 'б': 'литера',
 'в': 'литера',
 'вл': 'дом',
 'г': 'город',
 'гор': 'город',
 'город': 'город',
 'д': 'дом',
 'д-ня': 'деревня',
 'дер': 'деревня',
 'деревня': 'деревня',
 'дом': 'дом',
 'инд': 'индекс',
 'индекс': 'индекс',
 'к': 'корпус',
 'кабинет': 'квартира',
 'кв': 'квартира',
 'квартира': 'квартира',
 'ком': 'квартира',
 'комн': 'квартира',
 'комната': 'квартира',
 'копр': 'корпус',
 'кор': 'корпус',
 'корп': 'корпус',
 'корпус': 'корпус',
 'оф': 'офис',
 'офис': 'офис',
 'пом': 'квартира',
 'помещение': 'квартира',
 'поселок': 'с/п',
 'с': 'корпус',
 'сел': 'село',
 'село': 'село',
 'сельский': 'с/п',
 'сп': 'с/п',
 'стр': ['корпус', 'страна'],
 'страна': 'страна',
 'строен': 'корпус',
 'строение': 'корпус'}

In [660]:
shortables.update(inverdic(shortables))

In [669]:
inverdic(shortables)

{'б': {'большая', 'большой'},
 'б-й': {'большой'},
 'б-я': {'большая'},
 'бол': {'большая', 'большой'},
 'большая': {'большая'},
 'большой': {'большой'},
 'м': {'малая', 'малый'},
 'м-й': {'малый'},
 'м-я': {'малая'},
 'мал': {'малая', 'малый'},
 'малая': {'малая'},
 'малый': {'малый'},
 'с': {'средний'},
 'ср': {'средний'},
 'ср-ий': {'средний'},
 'ср-й': {'средний'},
 'средний': {'средний'}}

In [671]:
{**shortables, **inverdic(shortables)}

{'б': {'большая', 'большой'},
 'б-й': {'большой'},
 'б-я': {'большая'},
 'бол': {'большая', 'большой'},
 'большая': {'большая'},
 'большой': {'большой'},
 'м': {'малая', 'малый'},
 'м-й': {'малый'},
 'м-я': {'малая'},
 'мал': {'малая', 'малый'},
 'малая': {'малая'},
 'малый': {'малый'},
 'с': {'средний'},
 'ср': {'средний'},
 'ср-ий': {'средний'},
 'ср-й': {'средний'},
 'средний': {'средний'}}

In [667]:
shortables

{'б': {'большая', 'большой'},
 'б-й': {'большой'},
 'б-я': {'большая'},
 'бол': {'большая', 'большой'},
 'большая': {'б', 'б-я', 'бол', 'большая'},
 'большой': {'б', 'б-й', 'бол', 'большой'},
 'м': {'малая', 'малый'},
 'м-й': {'малый'},
 'м-я': {'малая'},
 'мал': {'малая', 'малый'},
 'малая': {'м', 'м-я', 'мал', 'малая'},
 'малый': {'м', 'м-й', 'мал', 'малый'},
 'с': {'средний'},
 'ср': {'средний'},
 'ср-ий': {'средний'},
 'ср-й': {'средний'},
 'средний': {'с', 'ср', 'ср-ий', 'ср-й', 'средний'}}

In [647]:
inverdic(shortables)

{'большая': {'б', 'б-я', 'бол', 'большая'},
 'большой': {'б', 'б-й', 'бол', 'большой'},
 'малая': {'м', 'м-я', 'мал', 'малая'},
 'малый': {'м', 'м-й', 'мал', 'малый'},
 'средний': {'с', 'ср', 'ср-ий', 'ср-й', 'средний'}}

In [650]:
shortables.update(inverdic(shortables))

In [651]:
shortables

{'б': {'большая', 'большой'},
 'б-й': {'большой'},
 'б-я': {'большая'},
 'бол': {'большая', 'большой'},
 'большая': {'б', 'б-я', 'бол', 'большая'},
 'большой': {'б', 'б-й', 'бол', 'большой'},
 'м': {'малая', 'малый'},
 'м-й': {'малый'},
 'м-я': {'малая'},
 'мал': {'малая', 'малый'},
 'малая': {'м', 'м-я', 'мал', 'малая'},
 'малый': {'м', 'м-й', 'мал', 'малый'},
 'с': {'средний'},
 'ср': {'средний'},
 'ср-ий': {'средний'},
 'ср-й': {'средний'},
 'средний': {'с', 'ср', 'ср-ий', 'ср-й', 'средний'}}

In [677]:
def fill(dic):
  newdic = dict()
  for key, value in dic.items():
    newdic[key] = value
    for item in value:
      try:
        newdic[item].add(key)
      except KeyError:
        newdic[item] = {key, *value}
  return newdic

In [678]:
fill(shortables)

{'б': {'б', 'б-я', 'бол', 'большая', 'большой'},
 'б-й': {'б', 'б-й', 'бол', 'большой'},
 'б-я': {'б', 'б-я', 'бол', 'большая'},
 'бол': {'б', 'б-я', 'бол', 'большая', 'большой'},
 'большая': {'б', 'б-я', 'бол', 'большая'},
 'большой': {'б', 'б-й', 'бол', 'большой'},
 'м': {'м', 'м-я', 'мал', 'малая', 'малый'},
 'м-й': {'м', 'м-й', 'мал', 'малый'},
 'м-я': {'м', 'м-я', 'мал', 'малая'},
 'мал': {'м', 'м-я', 'мал', 'малая', 'малый'},
 'малая': {'м', 'м-я', 'мал', 'малая'},
 'малый': {'м', 'м-й', 'мал', 'малый'},
 'с': {'с', 'ср', 'ср-ий', 'ср-й', 'средний'},
 'ср': {'с', 'ср', 'ср-ий', 'ср-й', 'средний'},
 'ср-ий': {'с', 'ср', 'ср-ий', 'ср-й', 'средний'},
 'ср-й': {'с', 'ср', 'ср-ий', 'ср-й', 'средний'},
 'средний': {'с', 'ср', 'ср-ий', 'ср-й', 'средний'}}

In [680]:
shortables = {'б': {'б', 'большая', 'большой'},
 'б-й': {'б', 'большой'},
 'б-я': {'б', 'большая'},
 'бол': {'б', 'большая', 'большой'},
 'большая': {'б', 'большая'},
 'большой': {'б', 'большой'},
 'м': {'м', 'малая', 'малый'},
 'м-й': {'м', 'малый'},
 'м-я': {'м', 'малая'},
 'мал': {'м', 'малая', 'малый'},
 'малая': {'м', 'малая'},
 'малый': {'м', 'малый'},
 'с': {'с', 'ср', 'средний'},
 'ср': {'с', 'ср', 'средний'},
 'ср-ий': {'с', 'ср', 'средний'},
 'ср-й': {'с', 'ср', 'средний'},
 'средний': {'с', 'ср', 'средний'}}

In [689]:
%%timeit 
for key in shortables.keys():
  key in "Москва, улица Большая Татарска, 35"

1.29 µs ± 11.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [694]:
%out 582

UsageError: Line magic function `%out` not found.


In [775]:
verify_address('МОСКВА, пр. Рязанский',  debug=True)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '36b4d93f-38cd-4b02-9d0d-c005f14e38eb',
    '_index': 'fias_full_text',
    '_score': 20.72171,
    '_source': {'aolevel': '7',
     'fullname': 'г Москва, пр-кт Рязанский',
     'guid': '36b4d93f-38cd-4b02-9d0d-c005f14e38eb',
     'region': 'Москва',
     'region_type': 'г',
     'street': 'Рязанский',
     'street_type': 'пр-кт'},
    '_type': 'address'}],
  'max_score': 20.72171,
  'total': 17199},
 'timed_out': False,
 'took': 10}

In [766]:
ref2["Исходный адрес"].iloc[1016]

'МОСКВА, 109542, ул. Рязанский, д. 86/1, стр. 3, оф. 6а'

In [ ]:
[map_types(x) for x in tokenize(ref3['Исходный адрес'].iloc[random.randint(0, 1040)])]

In [2]:
print(1)

1


In [12]:
search_dict("д", dic=sep_house_signs)

['дом']

In [30]:
set("абвгдежзиклмно")

{'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'к', 'л', 'м', 'н', 'о'}

In [42]:
map_types()
check_num()
tokenize()

TypeError: map_types() missing 1 required positional argument: 'token'

In [198]:
tokenize('МОСКВА, 109542, ул. Рязанский, Д. 86/1, стр. 3, оф. 6а', comma=True)

['МОСКВА',
 ',',
 '109542',
 ',',
 'ул',
 'Рязанский',
 ',',
 'Д',
 '86',
 '1',
 ',',
 'стр',
 '3',
 ',',
 'оф',
 '6',
 'а']

In [69]:
check_num([map_types(x) for x in Out[67]], -2)

['число']

In [125]:
[house_signs_inv.get(x, a(x)) for x in Out[67]]

['не распознано',
 'препинания',
 'не распознано',
 'препинания',
 'не распознано',
 'не распознано',
 'препинания',
 'дом',
 'число',
 'число',
 'препинания',
 'строение',
 'число',
 'препинания',
 'офис',
 'число',
 'литера']

In [126]:
for i, _ in enumerate(Out[67]):
  print(Out[67][i], " ==> ", Out[125][i])

МОСКВА  ==>  не распознано
,  ==>  препинания
109542  ==>  не распознано
,  ==>  препинания
ул  ==>  не распознано
Рязанский  ==>  не распознано
,  ==>  препинания
д  ==>  дом
86  ==>  число
1  ==>  число
,  ==>  препинания
стр  ==>  строение
3  ==>  число
,  ==>  препинания
оф  ==>  офис
6  ==>  число
а  ==>  литера


In [124]:
a = lambda x: "число" if x.isdigit() and len(x)<6 else "препинания" if x == ',' else "не распознано"

In [240]:
def longest_seq(A, item):
  array = (list(y) for (x,y) in itertools.groupby((enumerate(A)),operator.itemgetter(1)) if x == item)
  a = max(array, key=len)
  print(a)

In [239]:
longest_seq([1,2,0,0,3, 3, 3, 4,5,-1,0,2,-1,-3,0,0,0,0,0,0,0,0,2,-3,-4,-5,0,0,0], 0)

[(14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0)]


In [195]:
%timeit extract_house(Out[67])

18.6 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [123]:
# берём длиннейшую последовательность распознанного с приоритетом того что ближе к концу

In [156]:
set([-3, -4, -5])

{-5, -4, -3}

In [64]:
a[4] = ["улица"]

In [65]:
a

[['не распознано'],
 ['препинания'],
 ['не распознано'],
 ['не распознано'],
 ['улица'],
 ['препинания'],
 ['число']]

In [203]:
re.search('дом 3 строение 5', "Улица карамольская, Дом 3 Строение 5")

In [247]:
extract_house("Улица карамольская, Дом 3 Строение 5")

('Улица карамольская', ', Дом 3 Строение 5')

In [364]:
max(reversed([[2, 4], [3], [4, 6]]), key=len)

[4, 6]